In [7]:
'''
使用colab直接運行下載
'''
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html


In [12]:
import torch
from pytorch3d.structures import Volumes
from pytorch3d.renderer.implicit.renderer import  VolumeSampler
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

加載ray_bundle

In [13]:
checkpoint = torch.load('ray_sampling.pt')
ray_bundle = checkpoint.get('ray_bundle')

We then define a volume. The densities tensor has a shape of [10, 1, 64, 64, 50], where we have a batch of 10 volumes, and each volume is a grid of 64x64x50 nodes. Each node has one number to represent the density at the node. On the other hand, the colors tensor has a shape of [10, 3, 64, 64, 50], because each color needs three numbers to represent the RGB values:

然後我們定義一個體積。 密度張量的形狀為 [10, 1, 64, 64, 50]，其中我們有一批 10 個卷，每個卷是 64x64x50 節點的網格。 每個節點都有一個數字來表示該節點的密度。 另一方面，顏色張量的形狀為 [10, 3, 64, 64, 50]，因為每種顏色需要三個數字來表示 RGB 值：


In [15]:
batch_size = 10
densities = torch.zeros([batch_size,1,64,64,64]).to(device)
colors = torch.zeros(batch_size,3,64,64,64).to(device)
voxel_size = 0.1

volumes = Volumes(
    densities = densities,
    features= colors,
    voxel_size=voxel_size
)

我們需要根據體積定義volume_sampler。 在這裡，我們使用雙線性插值進行體積採樣。 然後可以透過將 ray_bundle 傳遞給volume_sampler 輕鬆獲得光線上點的密度和顏色：

In [16]:
volume_sampler = VolumeSampler(volumes=volumes,sample_mode='bilinear')
rays_densities,rays_features = volume_sampler(ray_bundle)
print('rays_densities shape = ',rays_densities.shape)
print('rays_features shape = ',rays_features.shape)

rays_densities shape =  torch.Size([10, 64, 64, 50, 1])
rays_features shape =  torch.Size([10, 64, 64, 50, 3])


8. Finally, let us save the densities and colors because we need to use them in the next section:

In [17]:
torch.save({'rays_densities':rays_densities,
            'rays_features':rays_features},'volume_sampling.pt')